# Cell 1 — Install & Imports

In [1]:
!pip install transformers datasets accelerate torch



  Attempting uninstall: fsspec

    Found existing installation: fsspec 2026.1.0

    Uninstalling fsspec-2026.1.0:

      Successfully uninstalled fsspec-2026.1.0

   -------- -------------------------------  3/14 [fsspec]
   -------------- -------------------------  5/14 [dill]
   ---------------------- -----------------  8/14 [yarl]
   ------------------------------- -------- 11/14 [aiohttp]
   ------------------------------- -------- 11/14 [aiohttp]
   ---------------------------------- ----- 12/14 [accelerate]
   ---------------------------------- ----- 12/14 [accelerate]
   ------------------------------------- -- 13/14 [datasets]
   ------------------------------------- -- 13/14 [datasets]
   ---------------------------------------- 14/14 [datasets]



In [1]:
import pandas as pd
import torch
from datasets import Dataset
from transformers import (
    DistilBertTokenizerFast,
    DistilBertForSequenceClassification,
    TrainingArguments,
    Trainer
)


#  Cell 2 — Load Data (Train / Val / Test)

In [2]:
train_df = pd.read_csv("../data/processed/train.csv")
val_df = pd.read_csv("../data/processed/val.csv")
test_df = pd.read_csv("../data/processed/test.csv")

train_df = train_df.rename(columns={"review": "text", "sentiment": "label"})
val_df = val_df.rename(columns={"review": "text", "sentiment": "label"})
test_df = test_df.rename(columns={"review": "text", "sentiment": "label"})


# Cell 3 — Convert to HuggingFace Dataset

In [3]:
train_ds = Dataset.from_pandas(train_df)
val_ds = Dataset.from_pandas(val_df)
test_ds = Dataset.from_pandas(test_df)


# Cell 4 — Tokenizer

In [4]:
tokenizer = DistilBertTokenizerFast.from_pretrained(
    "distilbert-base-uncased"
)

def tokenize(batch):
    return tokenizer(
        batch["text"],
        padding="max_length",
        truncation=True,
        max_length=256
    )

train_ds = train_ds.map(tokenize, batched=True)
val_ds = val_ds.map(tokenize, batched=True)
test_ds = test_ds.map(tokenize, batched=True)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

c:\Users\primu\anaconda3\envs\movie_sentiment_env\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\primu\.cache\huggingface\hub\models--distilbert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

Map:   0%|          | 0/40000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

# Cell 5 — Set Torch Format

In [5]:
columns = ["input_ids", "attention_mask", "label"]

train_ds.set_format(type="torch", columns=columns)
val_ds.set_format(type="torch", columns=columns)
test_ds.set_format(type="torch", columns=columns)


# Cell 6 — Load DistilBERT Model

In [6]:
model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2
)


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# 📌 Cell 7 — Training Configuration

In [8]:
training_args = TrainingArguments(
    output_dir="../models/bert_sentiment",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_dir="./logs",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss"
)


# ⚠️ 2 epochs is enough for IMDb on DistilBERT.

# 📌 Cell 8 — Metrics

In [9]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds)
    }


# Cell 9 — Trainer

In [10]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


C:\Users\primu\AppData\Local\Temp\ipykernel_28484\3533869841.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


# Cell 10 — Train Model

In [11]:
trainer.train()


c:\Users\primu\anaconda3\envs\movie_sentiment_env\lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.270300,0.244422,0.900000,0.902610
2,0.192000,0.269317,0.909200,0.910595


c:\Users\primu\anaconda3\envs\movie_sentiment_env\lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


TrainOutput(global_step=5000, training_loss=0.24686633453369142, metrics={'train_runtime': 20921.3714, 'train_samples_per_second': 3.824, 'train_steps_per_second': 0.239, 'total_flos': 5298695946240000.0, 'train_loss': 0.24686633453369142, 'epoch': 2.0})

# Cell 11 — Evaluate on Test Set

In [12]:
trainer.evaluate(test_ds)


c:\Users\primu\anaconda3\envs\movie_sentiment_env\lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'eval_loss': 0.24762700498104095,
 'eval_accuracy': 0.9014,
 'eval_f1': 0.9027037694888495,
 'eval_runtime': 267.0708,
 'eval_samples_per_second': 18.722,
 'eval_steps_per_second': 1.172,
 'epoch': 2.0}